# BT02: Ẩn tin mật trên ảnh gif (ảnh palette-based) bằng phương pháp LSB

Cập nhật ngày 09/12/2020 (ở TEST thứ 3 của hàm nhúng)

---
## Cách làm bài và nộp bài
&#9889; Bạn lưu ý là mình sẽ dùng chương trình hỗ trợ chấm bài nên bạn cần phải tuân thủ chính xác qui định mà mình đặt ra, nếu không rõ thì hỏi, chứ không nên tự tiện làm theo ý của cá nhân.

**Cách làm bài**

Bạn sẽ làm trực tiếp trên file notebook này. Đầu tiên, bạn điền họ tên và MSSV vào phần đầu file ở bên trên. Trong file, bạn làm bài ở những chỗ có ghi là:
```python
# YOUR CODE HERE
raise NotImplementedError()
```
hoặc đối với những phần code không bắt buộc thì là:
```python
# YOUR CODE HERE (OPTION)
```
hoặc đối với markdown cell thì là:
```markdown
YOUR ANSWER HERE
```
Tất nhiên, khi làm thì bạn xóa dòng `raise NotImplementedError()` đi.
Đối những phần yêu cầu code thì thường ở ngay phía dưới sẽ có một (hoặc một số) cell chứa các bộ test để giúp bạn biết đã code đúng hay chưa; nếu chạy cell này không có lỗi gì thì có nghĩa là qua được các bộ test. Trong một số trường hợp, các bộ test có thể sẽ không đầy đủ; nghĩa là, nếu không qua được test thì là code sai, nhưng nếu qua được test thì chưa chắc đã đúng hoàn toàn.

Trong khi làm bài, bạn có thể cho in ra màn hình, tạo thêm các cell để test. Nhưng khi nộp bài thì bạn xóa các cell mà bạn tự tạo, xóa hoặc comment các câu lệnh in ra màn hình. Bạn lưu ý <font color=red>không được tự tiện xóa các cell hay sửa code của Thầy</font> (trừ những chỗ được phép sửa như đã nói ở trên).

Trong khi làm bài, thường xuyên `Ctrl + S` để lưu lại bài làm của bạn, tránh mất mát thông tin.


*Nên nhớ mục tiêu chính ở đây là <font color=green>học, học một cách chân thật</font>. Bạn có thể thảo luận ý tưởng với bạn khác, nhưng <font color=green>code và bài làm phải là của bạn, dựa trên sự hiểu thật sự của bạn</font>. <font color=red>Nếu vi phạm thì sẽ bị 0 điểm cho toàn bộ môn học.</font>*

**Cách nộp bài**

Khi chấm bài, đầu tiên mình sẽ chọn `Kernel` - `Restart Kernel & Run All Cells`, để restart và chạy tất cả các cell trong notebook của bạn; do đó, trước khi nộp bài, bạn nên chạy thử `Kernel` - `Restart Kernel & Run All Cells` để đảm bảo mọi chuyện diễn ra đúng như mong đợi.

Sau đó, bạn tạo thư mục nộp bài theo cấu trúc sau:
- Thư mục `MSSV` (vd, nếu bạn có MSSV là 1234567 thì bạn đặt tên thư mục là `1234567`)
    - Thư mục `BT02`
        - File `BT02-AnTinMatTrenAnhGif_LSB.ipynb` (không cần nộp các file khác)

Cuối cùng, bạn nén thư mục `MSSV` này lại và nộp ở link trên moodle. <font color=red>Bạn lưu ý tuân thủ chính xác cấu trúc này.</font>

---
## Import

In [10]:
from bitarray import bitarray
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# YOUR CODE HERE (OPTION)

---
## Định nghĩa các hàm hỗ trợ (nếu cần)

In [11]:
# YOUR CODE HERE (OPTION)

---
## Hàm nhúng (7đ, trong đó: phần code 5đ, phần nhận xét và giải thích kết quả 2đ)

&#9889; Lưu ý:
- Khi tính toán bạn cần để ý kiểu dữ liệu. Ví dụ, nếu lấy 2 giá trị có kiểu dữ liệu `uint8` trừ cho nhau thì kết quả cũng sẽ có kiểu `uint8` và sẽ bị sai.
- Qui ước (để mình và bạn ra cùng kết quả): nếu trong bảng màu có nhiều màu thỏa điều kiện tìm kiếm thì lấy màu có chỉ số nhỏ nhất trong bảng màu. Nếu bạn có dùng hàm `sort` hay `argsort` của numpy thì bạn cần truyền tham số `kind="stable"`; nếu không thì các phần tử có giá trị giống nhau có thể sẽ bị thay đổi thứ tự tương đối với nhau trong quá trình sort.

In [15]:
def embed(msg_file, cover_img_file, stego_img_file):
    '''
    Nhúng tin mật vào ảnh gif (ảnh palette-base) bằng phương pháp của Fridrich 
    (xem file slide "06-AnTinMatTrenAnh2.pdf", trang 24).
    
    Các tham số:
        msg_file (str): Tên file chứa secret message.
        cover_img_file (str): Tên file chứa cover image.
        stego_img_file (str): Tên file chứa stego image (kết quả sau khi nhúng).
    Giá trị trả về:
        bool: True nếu nhúng thành không, False nếu không đủ chỗ để nhúng. 
    '''
    # Đọc cover img file
    cover_img = Image.open(cover_img_file)
    cover_pixels = np.array(cover_img)
    palette = cover_img.getpalette() # Kết quả là list các giá trị Red, Green, Blue của các màu 
                                     # trong bảng màu, 3 giá trị liên tiếp ứng với một màu
    palette = np.array(palette, dtype=np.uint8).reshape(1, -1, 3) # Reshape lại dưới dạng ảnh RGB
    #plt.figure(); plt.yticks([]); plt.imshow(palette, aspect=20) # Uncomment để xem bảng màu
    
    # YOUR CODE HERE
    #raise NotImplementedError()
    # Đọc msg file
    with open(msg_file, 'r') as f:
        msg = f.read()
    print(len(msg))
    
    # Chuyển msg thành msg bits
    msg_bits = bitarray()
    msg_bits.frombytes(msg.encode('utf-8'))
    print(len(msg_bits))
    
    # Kiểm xem có nhúng được không?
    capacity = cover_pixels.size * 2
    if len(msg_bits) + 1 > capacity:
        print('Cannot embed')
        return False
    
    # Thêm '100...' vào msg bits
    msg_bits.extend('1' + '0' * (capacity - len(msg_bits) - 1))
   
    # Nhúng msg bits vào cover pixels
    stego_pixels = np.empty_like(cover_pixels)
    for r in range(cover_pixels.shape[0]):
        for c in range(cover_pixels.shape[1]):
            b = (r * cover_pixels.shape[1] + c) * 2
            stego_pixels[r, c] = (cover_pixels[r, c]>>2<<2)\
                                + int(msg_bits[b:b+2].to01(), 2)
    
    
    # Ghi stego pixels cùng palette xuống file
    stego_img = Image.fromarray(stego_pixels)
    stego_img.putpalette(cover_img.getpalette())
    stego_img.save(stego_img_file)
    
    return True

In [16]:
# TEST 
result = embed('msg2.txt', 'lena.gif', 'lena_stego.gif')
assert result == False

34798
278384


AssertionError: 

In [17]:
# TEST             
result = embed('msg.txt', 'lena.gif', 'lena_stego.gif')
assert result == True
stego_img = Image.open('lena_stego.gif')
stego_pixels = np.array(stego_img)
stego_palette = stego_img.getpalette()
correct_stego_img = Image.open('correct_lena_stego.gif')
correct_stego_pixels = np.array(correct_stego_img)
correct_stego_palette = correct_stego_img.getpalette()
assert np.all(stego_pixels == correct_stego_pixels)
assert stego_palette == correct_stego_palette

28998
231984


AssertionError: 

In [ ]:
# TEST             
result = embed('msg.txt', 'baboon.gif', 'baboon_stego.gif')
assert result == True
stego_img = Image.open('baboon_stego.gif')
stego_pixels = np.array(stego_img)
stego_palette = stego_img.getpalette()
correct_stego_img = Image.open('correct_baboon_stego.gif')
correct_stego_pixels = np.array(correct_stego_img)
correct_stego_palette = correct_stego_img.getpalette()
assert np.all(stego_pixels == correct_stego_pixels)
assert stego_palette == correct_stego_palette

Xem các ảnh và so sánh tính vô hình khi nhúng trên ảnh "lena.gif" với khi nhúng trên ảnh "baboon.gif". Theo bạn, tại sao lại như vậy? (Gợi ý: trong hàm nhúng ở trên, bạn có thể tính và cho in ra giá trị trung bình của khoảng cách từ mỗi màu trong bảng màu tới màu gần nhất mà có (R+G+B)%2 khác với (R+G+B)%2 của mình)

YOUR ANSWER HERE

---
## Hàm rút trích (3đ)

In [20]:
def extract(stego_img_file, extr_msg_file):
    '''
    Hàm rút trích tin mật đã được nhúng vào ảnh bằng phương pháp của Fridrich.
    
    Các tham số:
        stego_img_file (str): Tên file chứa stego image.
        extr_msg_file (str): Tên file chứa secret message được rút trích.
    '''
    # YOUR CODE HERE
    # Đọc stego img file
    stego_pixels = np.array(Image.open(stego_img_file))
                      
    # Rút trích msg bits
    extr_msg_bits = bitarray()
   
    for pixel in stego_pixels.reshape(-1):
        extr_msg_bits.extend((np.binary_repr(pixel & (2**2-1), 2)))
    pixel = 10101100
    num_ = 2

    # Cắt đuôi '100...' ra khỏi msg bits
    extr_msg_bits = extr_msg_bits[:extr_msg_bits.to01().rfind('1')]
    
    # Chuyển msg bits thành msg
    extr_msg = extr_msg_bits.tobytes().decode('utf-8')
    
    # Ghi msg xuống file
    with open(extr_msg_file, 'w') as f:
        f.write(extr_msg)

In [21]:
# TEST
extract('correct_baboon_stego.gif', 'extr_msg.txt')
with open('extr_msg.txt', 'r') as f:
    extr_msg = f.read()
with open('msg.txt', 'r') as f:
    correct_extr_msg = f.read()
assert extr_msg == correct_extr_msg

UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 3-4: invalid continuation byte